In [1]:
import pandas as pd
from sqlalchemy import create_engine
import warnings

postgres_db = 'weatherinszeged'

## House price model

In [2]:
postgres_db='houseprices'
engine2 = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices_df = pd.read_sql_query('select * from houseprices',con=engine2)
engine2.dispose()

In [3]:
# select features  year build, sqfeet , lot area , full bath ,halfbath, overallcond, live area
fea_df = houseprices_df[['yearbuilt','lotarea','fullbath','halfbath','grlivarea','overallcond','saleprice']]

In [4]:
# bath and condition categorical
bath_df = pd.get_dummies(fea_df['fullbath'],dtype=int).rename(columns=lambda x:'fb_' +str(x))
new_df = fea_df.drop(['fullbath'], axis =1).merge(bath_df,left_index=True, right_index=True)

In [5]:
bath_df = pd.get_dummies(fea_df['halfbath'],dtype=int).rename(columns=lambda x:'hb_' +str(x))
new2_df = new_df.drop(['halfbath'], axis =1).merge(bath_df,left_index=True, right_index=True)

In [6]:
new2_df.head()

,yearbuilt,lotarea,grlivarea,overallcond,saleprice,fb_0,fb_1,fb_2,fb_3,hb_0,hb_1,hb_2
0,2003,8450,1710,5,208500,0,0,1,0,0,1,0
1,1976,9600,1262,8,181500,0,0,1,0,1,0,0
2,2001,11250,1786,5,223500,0,0,1,0,0,1,0
3,1915,9550,1717,5,140000,0,1,0,0,1,0,0
4,2000,14260,2198,5,250000,0,0,1,0,0,1,0


In [7]:

# `Y` is the target variable
Y = new2_df['saleprice']
# `X` is the feature set which includes the
# `is_male` and `is_smoker` variables
X = new2_df.drop(['saleprice'], axis =1)

In [13]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
# Fit an OLS model using scikit-learn
lrm = LinearRegression()
lrm.fit(X_train, y_train)


# You are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.6925319399003738
-----Test set statistics-----
R-squared of the model in the test set is: 0.7078845618659644
Mean absolute error of the prediction is: 28069.088761753683
Mean squared error of the prediction is: 1961176608.5607944
Root mean squared error of the prediction is: 44285.17368782462
Mean absolute percentage error of the prediction is: 15.73929590479116


In [23]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   yearbuilt    1460 non-null   int64
 1   lotarea      1460 non-null   int64
 2   grlivarea    1460 non-null   int64
 3   overallcond  1460 non-null   int64
 4   fb_0         1460 non-null   int64
 5   fb_1         1460 non-null   int64
 6   fb_2         1460 non-null   int64
 7   fb_3         1460 non-null   int64
 8   hb_0         1460 non-null   int64
 9   hb_1         1460 non-null   int64
 10  hb_2         1460 non-null   int64
dtypes: int64(11)
memory usage: 125.6 KB


This model's F-statistic is 369.4, and the associated p-value is very close to zero. This means that your features add some information to the reduced model, and your model is useful in explaining house prices.
AIC/BIC IS QUITE BIG , WE MAY NEED TO SELECT MORE FEATURES TO GET BETTER MODEL 
R SQARE IS .696 , MODEL IS SOME WHAT EXPLAINING THE HOUSE PRICE , ADDING OTHER VARIABLES MIGHT IMPROVE PERFORMANCE

In [24]:
from sklearn.linear_model import Ridge

# Fitting a ridge regression model. Here, `alpha` is the regularization
# parameter (usually called `lambda`). As `alpha` gets larger, parameter
# shrinkage grows more pronounced.
ridgeregr = Ridge(alpha=10**37) 
ridgeregr.fit(X_train, y_train)

# You are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [26]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=10**20.5) 
lassoregr.fit(X_train, y_train)

# You are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


R-squared of the model on the training set is: 0.0
-----Test set statistics-----
R-squared of the model on the test set is: -0.001618340746328606
Mean absolute error of the prediction is: 58023.64411709514
Mean squared error of the prediction is: 6724569139.943377
Root mean squared error of the prediction is: 82003.47029207592
Mean absolute percentage error of the prediction is: 36.65964810134902


# the model better fit is with no normalization 